In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [2]:
url = "http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario?codEstDE=&codEstATE=&tpEst=&nmEst=&nmRio=&codSubBacia=&codBacia=&nmMunicipio=&nmEstado=&sgResp=&sgOper=&telemetrica="
response = requests.get(url=url)

In [3]:
stations = pd.read_xml(response.content, xpath=".//Table")

In [5]:
stations_geo = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.Longitude, stations.Latitude))

In [6]:
# filter outliers
stations_geo = stations_geo[stations_geo.Longitude > -100]

In [7]:
# get Brazilian boundaries
brazil = gpd.read_file("/pfs/work7/workspace/scratch/tu_zxobe27-master_thesis/data/misc/gadm_410-BRA.geojson", engine="pyogrio")

In [14]:
stations_geo["in_bounds"] = stations_geo.intersects(brazil.iloc[0].geometry)

In [26]:
stations_geo.to_feather("/pfs/work7/workspace/scratch/tu_zxobe27-master_thesis/data/water_quality/stations.feather")

---

In [1]:
import os
import requests
from time import sleep
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_binary  # Adds chromedriver binary to path

In [2]:
# Check if the queries.json file already exists in the specified path
if not os.path.exists("/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/queries.json"):
    # Load geographic data of stations from a Feather file
    stations_geo = gpd.read_feather("/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/stations.feather")
    
    # Filter stations where 'TipoEstacaoQualAgua' equals 1 and select only the 'Codigo' column
    stations_to_query = stations_geo.loc[stations_geo["TipoEstacaoQualAgua"] == 1, ["Codigo"]]
    
    # Initialize a new column 'success' with None values to track query success
    stations_to_query.loc[:, "success"] = None
    
    # Reset the index for a clean DataFrame and set 'Codigo' as the new index
    stations_to_query = stations_to_query.reset_index(drop=True).set_index("Codigo")
    
    # List all files in the raw data directory
    existing_files = os.listdir("/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/raw")
    
    # Extract numeric IDs from the file names and filter unique IDs
    existing_file_ids = pd.Series(existing_files).str.extract(r"(^\d*)").squeeze().astype(int).unique()
    
    # Update the 'success' status for stations where corresponding raw data files exist
    stations_to_query.loc[existing_file_ids, "success"] = True
    
    # Save the DataFrame to a JSON file for later use
    stations_to_query.to_json("/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/queries.json")


In [3]:
stations_to_query = pd.read_json("/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/queries.json")

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')  # Bypass OS security model
options.add_argument('--disable-gpu')  # applicable to windows os only
options.add_argument('start-maximized')  # start maximized
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
#options.add_argument('--headless')

downloadPath = "/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/raw"
prefs = {}
os.makedirs(downloadPath, exist_ok=True)
prefs["profile.default_content_settings.popups"]=0
prefs["download.default_directory"]=downloadPath
options.add_experimental_option("prefs", prefs)
# Initialize the driver
driver = webdriver.Chrome(options=options)

<selenium.webdriver.remote.webelement.WebElement (session="9f9035660aacb4d5b8a29d8f3231fed0", element="f.535B3A096E99E046028C6D262C7B8930.d.6B0BA2F47F50E3EC83B8796D88AA105E.e.490")>

In [11]:
def download_by_ID(id, driver):
    n_tries = 0
    while n_tries < 5:
        n_tries += 1
        
        if n_tries == 3:
            driver.get("https://www.snirh.gov.br/hidroweb/serieshistoricas")
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//button[@type="reset"]')))
        
        try:
            # clear field
            driver.find_element(By.XPATH, '//button[@type="reset"]').click()
            sleep(.5)
            # enter ID
            driver.find_element(By.XPATH, '//*[@name="codigoEstacao"]').send_keys(str(id))
            sleep(.5)
            # click to search
            driver.find_element(By.XPATH, '//button[@color="primary"]').click()
            # wait for results to load
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//td[contains(@class, "mat-column-csv")]/button'))
            )
            # download csv
            element.click()
            sleep(1)
            return True
        except:
            sleep(1)
    return False

In [12]:
driver.get("https://www.snirh.gov.br/hidroweb/serieshistoricas")
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//button[@type="reset"]')))

for idx in tqdm(stations_to_query[stations_to_query.success.isna()].index):
    stations_to_query.loc[idx, "success"] = download_by_ID(idx, driver)

  6%|▌         | 388/6368 [1:18:24<20:08:31, 12.13s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.119)
Stacktrace:
0   chromedriver                        0x0000000107db57c8 chromedriver + 5916616
1   chromedriver                        0x0000000107dad3e3 chromedriver + 5882851
2   chromedriver                        0x0000000107873a27 chromedriver + 404007
3   chromedriver                        0x000000010784a29e chromedriver + 234142
4   chromedriver                        0x00000001078ea1bd chromedriver + 889277
5   chromedriver                        0x00000001079001a8 chromedriver + 979368
6   chromedriver                        0x00000001078e1613 chromedriver + 853523
7   chromedriver                        0x00000001078b2303 chromedriver + 660227
8   chromedriver                        0x00000001078b2c4e chromedriver + 662606
9   chromedriver                        0x0000000107d736a0 chromedriver + 5645984
10  chromedriver   

In [13]:
stations_to_query.to_json("/Users/felixschulz/Library/CloudStorage/OneDrive-Personal/Dokumente/Uni/Masterthesis/data/water_quality/queries.json")

In [82]:
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.119)
Stacktrace:
0   chromedriver                        0x000000010117a7c8 chromedriver + 5916616
1   chromedriver                        0x00000001011723e3 chromedriver + 5882851
2   chromedriver                        0x0000000100c38a27 chromedriver + 404007
3   chromedriver                        0x0000000100c0f29e chromedriver + 234142
4   chromedriver                        0x0000000100caf1bd chromedriver + 889277
5   chromedriver                        0x0000000100cb600c chromedriver + 917516
6   chromedriver                        0x0000000100ca6838 chromedriver + 854072
7   chromedriver                        0x0000000100c77303 chromedriver + 660227
8   chromedriver                        0x0000000100c77c4e chromedriver + 662606
9   chromedriver                        0x00000001011386a0 chromedriver + 5645984
10  chromedriver   

In [18]:
import requests
import re
import pandas as pd

def get_cboTipoReg(option):
    cboTipoReg_values = {
        "Cota": 8, "Cotas": 8, "Vazao": 9, "Vazoes": 9,
        "Chuva": 10, "Chuvas": 10, "Clima": 11, "Qualidade": 12,
        "Resumo": 13, "Sedimento": 14, "Sedimentos": 14, "Perfil": 16
    }
    return {"cboTipoReg": cboTipoReg_values[option]}

def hidroweb_url(station_code):
    return f"http://hidroweb.ana.gov.br/Estacao.asp?Codigo={station_code}&CriaArq=true&TipoArq=1"

def hidroweb_post(url, data):
    response = requests.post(url, data=data)
    return response.text

def find_station_options(content):
    options = re.findall(r'option value="(\d+)">([^<]+)</option>', content)
    return pd.DataFrame(options, columns=['option_num', 'option'])

def extract_metadata(content):
    # Extracting metadata involves parsing the HTML content
    # Simplified example: Assume metadata is extracted as follows:
    data = {
        'station': re.search(r"Station Code: (\d+)", content).group(1),
        'longitude': float(re.search(r"Longitude: ([\d.-]+)", content).group(1)),
        'latitude': float(re.search(r"Latitude: ([\d.-]+)", content).group(1))
    }
    return pd.DataFrame([data])

def download_hydroweb_data(station, option, dest_dir=".", verbose=True):
    url = hidroweb_url(station)
    data = get_cboTipoReg(option)
    content = hidroweb_post(url, data)
    
    if "not found in database" in content:
        print("Station not found")
        return pd.DataFrame(columns=['station', 'file', 'longitude', 'latitude'])
    
    station_options = find_station_options(content)
    selected_option = station_options.query('option == @option')

    metadata = extract_metadata(content)
    
    if verbose:
        print("Metadata extracted:", metadata)
    
    return metadata

# Example usage:
download_hydroweb_data("3253005", "Chuva")

ConnectionError: HTTPConnectionPool(host='hidroweb.ana.gov.br', port=80): Max retries exceeded with url: /Estacao.asp?Codigo=3253005&CriaArq=true&TipoArq=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1501a914d1f0>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [20]:
response = requests.get(f"https://www.snirh.gov.br/hidroweb/rest/api/estacaotelemetrica?id={10300000}")

In [28]:
response = requests.get(f"https://www.snirh.gov.br/hidroweb/rest/api/documento/gerarTelemetricas?codigosEstacoes=193142390&tipoArquivo=1&periodoInicial=2022-02-11T03:00:00.000Z&periodoFinal=2022-02-12T03:00:00.000Z")

In [29]:
response.content

b'[{"id":193142390,"codigoEstacao":"56696000","codigoAdicional":"56696000","aneelPlu":"1942029","aneelFlu":"56696000","nome":"MARIO DE CARVALHO","latitude":-19.5247,"longitude":-42.6408,"altitude":232,"ultimaAtualizacao":"2023-06-27T03:00:00.000+0000","baciaCodigo":5,"codigoNomeBacia":"5 - Atl\xc3\xa2ntico, Trecho Leste","codigoNomeSubBacia":"56 - RIO DOCE","nomeRio":"RIO PIRACICABA","nomeEstado":"MINAS GERAIS","nomeMunicipio":"TIM\xc3\x93TEO","subBaciaCodigo":56,"rioCodigo":56180000,"estadoCodigo":17,"municipioCodigo":17686000,"responsavelCodigo":1,"operadoraCodigo":1,"menorDataPeriodo":"2010-01-26T21:00:00.000+0000","maiorDataPeriodo":"2024-05-04T10:30:00.000+0000","responsavelSigla":"ANA","operadoraSigla":"CPRM","tipoEstacao":"Hidrol\xc3\xb3gica","operando":1,"codigoNome":"193142390 - MARIO DE CARVALHO","medicoes":[{"id":{"horEstacao":193142390,"horDataHora":"2022-02-11T03:00:00.000+0000"},"horQChuva":0.0,"horChuva":2.2,"horQNivelAdotado":0.0,"horNivelAdotado":380.0,"horQVazao":0.0,

In [4]:
url = f"http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica?codEstacao={10300000}&dataInicio=&dataFim=&tipoDados=2&nivelConsistencia="	
response = requests.get(url=url)

In [12]:
df = pd.read_xml(response.content, xpath=".//SerieHistorica")

In [ ]:
http://hidroweb.ana.gov.br/Estacao.asp?Codigo=XXXXXXXX&CriaArq=true&TipoArq=1

In [13]:
df.columns

Index(['id', 'rowOrder', 'EstacaoCodigo', 'NivelConsistencia', 'DataHora',
       'MediaDiaria', 'TipoMedicaoCotas', 'Maxima', 'Minima', 'Media',
       'DiaMaxima', 'DiaMinima', 'MaximaStatus', 'MinimaStatus', 'MediaStatus',
       'MediaAnualStatus', 'Cota01', 'Cota02', 'Cota03', 'Cota04', 'Cota05',
       'Cota06', 'Cota07', 'Cota08', 'Cota09', 'Cota10', 'Cota11', 'Cota12',
       'Cota13', 'Cota14', 'Cota15', 'Cota16', 'Cota17', 'Cota18', 'Cota19',
       'Cota20', 'Cota21', 'Cota22', 'Cota23', 'Cota24', 'Cota25', 'Cota26',
       'Cota27', 'Cota28', 'Cota29', 'Cota30', 'Cota31', 'DataIns',
       'Cota01Status', 'Cota02Status', 'Cota03Status', 'Cota04Status',
       'Cota05Status', 'Cota06Status', 'Cota07Status', 'Cota08Status',
       'Cota09Status', 'Cota10Status', 'Cota11Status', 'Cota12Status',
       'Cota13Status', 'Cota14Status', 'Cota15Status', 'Cota16Status',
       'Cota17Status', 'Cota18Status', 'Cota19Status', 'Cota20Status',
       'Cota21Status', 'Cota22Status', 'Cot

In [17]:
df.Cota18

0       1471.0
1       1471.0
2       1471.0
3       1471.0
4       1478.0
         ...  
1247     456.0
1248       NaN
1249       NaN
1250       NaN
1251       NaN
Name: Cota18, Length: 1252, dtype: float64

In [12]:
df.columns

Index(['id', 'rowOrder', 'EstacaoCodigo', 'NivelConsistencia', 'DataHora',
       'MediaDiaria', 'TipoMedicaoCotas', 'Maxima', 'Minima', 'Media',
       'DiaMaxima', 'DiaMinima', 'MaximaStatus', 'MinimaStatus', 'MediaStatus',
       'MediaAnualStatus', 'Cota01', 'Cota02', 'Cota03', 'Cota04', 'Cota05',
       'Cota06', 'Cota07', 'Cota08', 'Cota09', 'Cota10', 'Cota11', 'Cota12',
       'Cota13', 'Cota14', 'Cota15', 'Cota16', 'Cota17', 'Cota18', 'Cota19',
       'Cota20', 'Cota21', 'Cota22', 'Cota23', 'Cota24', 'Cota25', 'Cota26',
       'Cota27', 'Cota28', 'Cota29', 'Cota30', 'Cota31', 'DataIns',
       'Cota01Status', 'Cota02Status', 'Cota03Status', 'Cota04Status',
       'Cota05Status', 'Cota06Status', 'Cota07Status', 'Cota08Status',
       'Cota09Status', 'Cota10Status', 'Cota11Status', 'Cota12Status',
       'Cota13Status', 'Cota14Status', 'Cota15Status', 'Cota16Status',
       'Cota17Status', 'Cota18Status', 'Cota19Status', 'Cota20Status',
       'Cota21Status', 'Cota22Status', 'Cot

In [18]:
stations_geo[stations_geo.in_bounds]

,id,rowOrder,BaciaCodigo,SubBaciaCodigo,RioCodigo,RioNome,EstadoCodigo,nmEstado,MunicipioCodigo,nmMunicipio,...,TipoRedeQualAgua,TipoRedeClasseVazao,UltimaAtualizacao,Operando,Descricao,NumImagens,DataIns,DataAlt,geometry,in_bounds
0,Table1,0,3,32,NaN,None,5,PARÁ,5063000,SALINÓPOLIS,...,0.0,0.0,2024-02-21 00:00:00,0,None,0.0,None,2024-02-21 00:00:00,POINT (-47.55000 -0.65000),True
1,Table2,1,3,32,NaN,None,5,PARÁ,5043000,MARACANÃ,...,0.0,0.0,2005-03-29 00:00:00,1,None,0.0,None,None,POINT (-47.50000 -0.83333),True
2,Table3,2,3,32,NaN,None,5,PARÁ,5063000,SALINÓPOLIS,...,0.0,0.0,2024-01-15 00:00:00,1,AUTOMÁTICA - IRIS-1,1.0,None,2024-01-15 00:00:00,POINT (-47.35360 -0.62310),True
3,Table4,3,3,32,NaN,None,5,PARÁ,5029000,CURUÇA,...,0.0,0.0,2018-06-18 00:00:00,1,AUTOMÁTICA - AGSOLV,2.0,None,2018-06-18 00:00:00,POINT (-47.85360 -0.73750),True
4,Table5,4,3,32,NaN,None,5,PARÁ,5061000,PRIMAVERA,...,0.0,0.0,2023-11-07 00:00:00,1,None,1.0,None,2023-11-07 00:00:00,POINT (-47.09940 -0.92940),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38055,Table38056,38055,8,88,88100000.0,CANAL DE SÃO GONÇALO,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,0.0,1.0,2018-02-02 00:00:00,0,None,0.0,None,2018-02-02 00:00:00,POINT (-52.33330 -31.78220),True
38056,Table38057,38056,8,88,88100000.0,CANAL DE SÃO GONÇALO,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,0.0,0.0,2017-03-28 00:00:00,0,None,0.0,None,2017-03-28 00:00:00,POINT (-52.22810 -31.78640),True
38057,Table38058,38057,8,88,88105000.0,ARROIO PELOTAS,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,3.0,0.0,2018-02-01 00:00:00,1,None,0.0,2015-04-27 00:00:00,2018-02-01 00:00:00,POINT (-52.28310 -31.75750),True
38058,Table38059,38058,8,88,88105000.0,ARROIO PELOTAS,24,RIO GRANDE DO SUL,24144000,PELOTAS,...,3.0,1.0,2018-12-06 00:00:00,1,None,0.0,None,2018-12-06 00:00:00,POINT (-52.46250 -31.57360),True


In [33]:
stations_geo.within(brazil.unary_union)

In [30]:
stations_geo[stations_geo.intersects(brazil.geometry.iloc[0])].plot()

KeyboardInterrupt: 

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'satellite-cloud-properties',
    {
        'format': 'tgz',
        'product_family': 'clara_a3',
        'origin': 'eumetsat',
        'variable': 'cloud_fraction',
        'year': [
            '1979', '1980', '1981',
            '1982', '1983', '1984',
            '1985', '1986', '1987',
            '1988', '1989', '1990',
            '1991', '1992', '1993',
            '1994', '1995', '1996',
            '1997', '1998', '1999',
            '2000', '2001', '2002',
            '2003', '2004', '2005',
            '2006', '2007', '2008',
            '2009', '2010', '2011',
            '2012', '2013', '2014',
            '2015', '2016', '2017',
            '2018', '2019', '2020',
        ],
        'time_aggregation': 'monthly_mean',
        'climate_data_record_type': 'thematic_climate_data_record',
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'area': [
            90, -180, -90,
            180,
        ],
    },
    'download.tar.gz')